In [17]:
import pandas as pd

In [18]:
chunksize = 1000000
df = pd.read_csv('D:/WebML/Recommender/data/animelist.csv', chunksize=chunksize)

In [19]:
watching_chunks = []
completed_chunks = []

for chunk in df:
    # 데이터 타입 최적화
    chunk['user_id'] = chunk['user_id'].astype('int32')
    chunk['anime_id'] = chunk['anime_id'].astype('int32')
    chunk['watching_status'] = chunk['watching_status'].astype('int8')
    chunk['rating'] = chunk['rating'].astype('float32')
    chunk['watched_episodes'] = chunk['watched_episodes'].astype('int16')

    # watching_status에 따라 watching과 completed 나누기
    watching_chunk = chunk[chunk['watching_status'] == 1].groupby('user_id')['anime_id'].apply(lambda x: ','.join(map(str, x))).reset_index()
    completed_chunk = chunk[chunk['watching_status'] == 2].groupby('user_id')['anime_id'].apply(lambda x: ','.join(map(str, x))).reset_index()

    watching_chunks.append(watching_chunk)
    completed_chunks.append(completed_chunk)

In [20]:
# 청크를 결합
watching = pd.concat(watching_chunks).groupby('user_id')['anime_id'].apply(lambda x: ','.join(x)).reset_index()
completed = pd.concat(completed_chunks).groupby('user_id')['anime_id'].apply(lambda x: ','.join(x)).reset_index()

# 결과를 user_id에 따라 병합하기
result = pd.merge(watching, completed, on='user_id', how='outer', suffixes=('_watching', '_completed'))

# 열 이름 변경
result.columns = ['user_id', 'watching', 'completed']

# 결측값을 빈 문자열로 대체
result['watching'] = result['watching'].fillna('')
result['completed'] = result['completed'].fillna('')

In [21]:
len(result)

324688

In [22]:
result.head()

,user_id,watching,completed
0,0,"67,6702,242,4898,21,24,2104,4722,6098,3125,481","68,1689,2913,1250,356,121,430,1829,1571,578,43..."
1,1,"37403,9989,24833,7674,34572,34566,2167,40852,1...","22199,6547,9919,5081,31043,31964,33486,36456,3..."
2,2,"24833,235,36721,40956,31933,5042,7593,21,35446...","5762,31580,33253,35028,9513,368,11633,31964,33..."
3,3,"2593,1604,14749,6114,199,849,33352,23283","31646,32998,25397,12291,1292,34881,32323,22199..."
4,5,"530,38680","36904,5112,6287,8676,66,36873,6645,226,120,332..."


In [24]:
result.to_csv('wc_list.csv',index=False,encoding='utf-8-sig')

In [28]:
watching_list = result[result['user_id'] == 0]['watching'].tolist()

In [29]:
watching_list

['67,6702,242,4898,21,24,2104,4722,6098,3125,481']